<a href="https://colab.research.google.com/github/grbarbosa3/DataAnalysis_App_Users/blob/main/App_Events_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APP Events Analysis

### Libraries

In [ ]:
import pandas as pd
import datetime


import matplotlib.pyplot as plt
import seaborn as sns

### Importing data

In [ ]:
event_df = pd.read_csv("/content/event_data.csv")

In [ ]:
event_df

,user_id,event_date,event_type,purchase_amount
0,c40e6a,2019-07-29 00:02:15,registration,NaN
1,a2b682,2019-07-29 00:04:46,registration,NaN
2,9ac888,2019-07-29 00:13:22,registration,NaN
3,93ff22,2019-07-29 00:16:47,registration,NaN
4,65ef85,2019-07-29 00:19:23,registration,NaN
...,...,...,...,...
79737,930c23,2019-09-01 23:57:41,simple_event,NaN
79738,a84999,2019-09-01 23:57:50,simple_event,NaN
79739,175e4d,2019-09-01 23:59:40,simple_event,NaN
79740,1c2210,2019-09-01 23:59:51,simple_event,NaN


In [ ]:
 # Verifying the purchase amount counts.

event_df.purchase_amount.value_counts(dropna=False)


NaN     73535
20.0     1291
50.0     1247
40.0     1233
30.0     1229
10.0     1207
Name: purchase_amount, dtype: int64

In [ ]:
# Verifying the null values in columns.

event_df.isna().sum()

user_id                0
event_date             0
event_type             0
purchase_amount    73535
dtype: int64

In [ ]:
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79742 entries, 0 to 79741
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          79742 non-null  object 
 1   event_date       79742 non-null  object 
 2   event_type       79742 non-null  object 
 3   purchase_amount  6207 non-null   float64
dtypes: float64(1), object(3)
memory usage: 2.4+ MB


In [ ]:
# Transforming the event_date column to datetime format.

event_df['event_date'] = pd.to_datetime(event_df['event_date'], format = '%Y-%m-%d %H:%M:%S')


In [ ]:
# Adjusting the week number of the year of 2019.

event_df['week'] = [int(i) + 1 for i in event_df['event_date'].dt.strftime('%W')]

In [ ]:
event_df['user_id'].value_counts()

aef0ea    24
e43583    24
2c8868    23
5fb555    23
8c9156    23
          ..
6031d5     1
3f5c29     1
615a35     1
01d643     1
48b765     1
Name: user_id, Length: 9996, dtype: int64

In [ ]:
event_df.head()

,user_id,event_date,event_type,purchase_amount,week
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31
1,a2b682,2019-07-29 00:04:46,registration,NaN,31
2,9ac888,2019-07-29 00:13:22,registration,NaN,31
3,93ff22,2019-07-29 00:16:47,registration,NaN,31
4,65ef85,2019-07-29 00:19:23,registration,NaN,31


### Registration cohort by week

In [ ]:
# Selecting the registration event_type.

registration_df = event_df.query('event_type == "registration"')

In [ ]:
# Getting the number of registration for week 33.

registration_df = registration_df[['user_id', 'week']]
registration_df
print("Registration cohort for the week 33:\n", registration_df[registration_df['week'] == 33].value_counts().sum())

Registration cohort for the week 33:
 2045


### Week lifetime

Getting the week number which represents the lifetime of each user based on event occurence.

In [ ]:
# Transforming the week column to week registration for further merge in event_df, to calculate the lifetime.

registration_df.columns = ['user_id', 'week_registration']

In [ ]:
# Merging both DataFrames.

event_df = event_df.merge(registration_df, how='left', on='user_id')

In [ ]:
# Weekly lifetime column creation.

event_df['weekly_lifetime'] = event_df['week'] - event_df['week_registration']

In [ ]:
event_df

,user_id,event_date,event_type,purchase_amount,week,week_registration,weekly_lifetime
0,c40e6a,2019-07-29 00:02:15,registration,NaN,31,31,0
1,a2b682,2019-07-29 00:04:46,registration,NaN,31,31,0
2,9ac888,2019-07-29 00:13:22,registration,NaN,31,31,0
3,93ff22,2019-07-29 00:16:47,registration,NaN,31,31,0
4,65ef85,2019-07-29 00:19:23,registration,NaN,31,31,0
...,...,...,...,...,...,...,...
79737,930c23,2019-09-01 23:57:41,simple_event,NaN,35,32,3
79738,a84999,2019-09-01 23:57:50,simple_event,NaN,35,33,2
79739,175e4d,2019-09-01 23:59:40,simple_event,NaN,35,32,3
79740,1c2210,2019-09-01 23:59:51,simple_event,NaN,35,33,2


### Registration cohort table

Creating a cohort 

In [ ]:
# Creating the table for registration cohort.

event_cohorts = event_df.pivot_table(index = ['event_type','week_registration','weekly_lifetime'], aggfunc = {'user_id':'nunique'}).reset_index()

In [ ]:
event_cohorts

,event_type,week_registration,weekly_lifetime,user_id
0,purchase,31,0,292
1,purchase,31,1,580
2,purchase,31,2,340
3,purchase,31,3,185
4,purchase,31,4,64
5,purchase,32,0,329
6,purchase,32,1,601
7,purchase,32,2,357
8,purchase,32,3,188
9,purchase,33,0,295


In [ ]:
# Sorting values for weekly registration.

cohorts_lifetime = event_cohorts.sort_values(by=['week_registration'])

In [ ]:
cohorts_lifetime

,event_type,week_registration,weekly_lifetime,user_id
0,purchase,31,0,292
20,simple_event,31,0,1490
23,simple_event,31,3,695
24,simple_event,31,4,290
15,registration,31,0,1975
21,simple_event,31,1,1824
22,simple_event,31,2,1228
4,purchase,31,4,64
3,purchase,31,3,185
2,purchase,31,2,340


In [ ]:
# Getting the lifetime that equals to 0, when a cohort is formed.

cohorts_lifetime = event_cohorts[event_cohorts['weekly_lifetime'] == 0]

In [ ]:
# Separating the columns for the lifetime cohort.

cohorts_lifetime = cohorts_lifetime[['event_type','week_registration', 'user_id']]

In [ ]:
# Changing the user_id column name so it can be used in the merging for retention rate table

cohorts_lifetime = cohorts_lifetime.rename(columns={'user_id':'user_cohort'})

In [ ]:
cohorts_lifetime

,event_type,week_registration,user_cohort
0,purchase,31,292
5,purchase,32,329
9,purchase,33,295
12,purchase,34,281
14,purchase,35,337
15,registration,31,1975
16,registration,32,1952
17,registration,33,2045
18,registration,34,1974
19,registration,35,2050


### Retention Rate

In [ ]:
# Merging the tables of event type cohorts and cohorts of lifetime.

event_cohorts = event_cohorts.merge(cohorts_lifetime, on='week_registration')

In [ ]:
# Getting the retention rate

event_cohorts['retention_rate'] = event_cohorts['user_id'] / event_cohorts['user_cohort']

In [ ]:
# Retention rate table

retention_df = event_cohorts.pivot_table(index='week_registration',
                                         columns = 'weekly_lifetime',
                                         values = 'retention_rate',
                                         aggfunc = 'sum')

In [ ]:
retention_df

weekly_lifetime,0,1,2,3,4
week_registration,,,,,
31,17.290193,11.063515,7.216136,4.049872,1.629153
32,15.902080,10.182911,6.805515,3.707104,NaN
33,17.609948,11.265121,7.859807,NaN,NaN
34,17.750197,11.235321,NaN,NaN,NaN
35,16.140519,NaN,NaN,NaN,NaN


In [ ]:
print("Week 3 for cohort ID 32: %.2f" % retention_df.iloc[1,3],"%")

Week 3 for cohort ID 32: 3.71 %


### Purchase analysis

In [ ]:
# Creating the Purchase DataFrame

purchase_df = event_df[['event_type','purchase_amount', 'weekly_lifetime', 'user_id','week_registration']]

In [ ]:
# Getting the non null purchase values

purchase_df[purchase_df['purchase_amount'].notna()].sort_values(by=['user_id'])

,event_type,purchase_amount,weekly_lifetime,user_id,week_registration
36262,purchase,10.0,0,00049f,33
25016,purchase,50.0,2,00082c,31
59475,purchase,30.0,1,001d5d,34
36886,purchase,30.0,0,002195,33
15582,purchase,30.0,0,0025c1,32
...,...,...,...,...,...
4807,purchase,40.0,0,ffb3f3,31
45358,purchase,50.0,2,ffbe61,32
29466,purchase,30.0,1,ffc66a,32
13851,purchase,50.0,1,ffc88b,31


In [ ]:
# Purchase table

purchase_df_table = event_df.pivot_table(index='week_registration',
                                         columns = 'weekly_lifetime',
                                         values = 'purchase_amount',
                                         aggfunc = 'mean')

purchase_df_table

weekly_lifetime,0,1,2,3,4
week_registration,,,,,
31,28.132911,29.511494,30.525000,29.142857,30.454545
32,30.222841,29.773692,29.928741,29.528302,NaN
33,30.689655,30.519663,29.171975,NaN,NaN
34,30.776699,30.929134,NaN,NaN,NaN
35,30.721925,NaN,NaN,NaN,NaN


In [ ]:
print("Week 3 for purchase cohort ID 31: %.2f" % purchase_df_table.iloc[0,3],"%")

Week 3 for purchase cohort ID 31: 29.14 %


In [ ]:
first_purchase_df = purchase_df.query('event_type == "purchase"')

In [ ]:
# Sorting the tables by week registration

first_purchase_df = first_purchase_df.sort_values('week_registration')

In [ ]:
# Getting the first purchase of each user

first_purchase_df.drop_duplicates(['user_id'], keep='first')

,event_type,purchase_amount,weekly_lifetime,user_id,week_registration
146,purchase,10.0,0,51dc02,31
54396,purchase,50.0,3,436325,31
18795,purchase,20.0,1,266dcd,31
18786,purchase,40.0,1,f6724c,31
18780,purchase,40.0,1,d67c85,31
...,...,...,...,...,...
72118,purchase,10.0,0,8638da,35
72067,purchase,40.0,0,33bc59,35
72064,purchase,20.0,0,8ab2c0,35
59969,purchase,30.0,0,22208f,35


In [ ]:
# Getting median between registration and first purchase

seconds = int(first_purchase_df.weekly_lifetime.median() * 7 * 24 * 60 * 60) 

In [ ]:
print(" The median time between user registration and first purchase is:",seconds,"seconds")

 The median time between user registration and first purchase is: 604800 seconds
